In [1]:
from transformers import BertTokenizer, BertForQuestionAnswering

# 加載預訓練的 BERT 模型和分詞器
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# from sklearn.model_selection import train_test_split
# from datasets import Dataset

# # 假設您已經有了知識圖譜數據
# data = [
#     {"Name": "Entry Barriers", "Definition": "Factors that deter or prevent new competitors from entering a market."},
#     {"Name": "Natural Barriers", "Definition": "Barriers due to market characteristics or geographical constraints."},
#     {"Name": "Artificial Barriers", "Definition": "Barriers created by firms or legal structures to restrict entry."},
#     {"Name": "Strategic Barriers", "Definition": "Barriers created intentionally by existing firms to deter competition."},
#     {"Name": "Economies of Scale", "Definition": "Cost advantages due to increased production, making it harder for new entrants to compete."},
#     {"Name": "High Initial Capital Requirements", "Definition": "Significant upfront investments required to enter a market."},
#     {"Name": "Geographic Constraints", "Definition": "Barriers arising from location-specific factors such as access to resources."},
#     {"Name": "Patent Protection", "Definition": "Legal barriers that prevent competitors from using patented technologies or processes."},
#     {"Name": "Brand Loyalty", "Definition": "Consumer preference for existing brands, reducing new entrants' market share."},
#     {"Name": "Government Regulations", "Definition": "Legal or administrative restrictions that limit market entry."},
#     {"Name": "Predatory Pricing", "Definition": "Deliberate underpricing to drive out competitors or prevent new entry."},
#     {"Name": "Network Effects", "Definition": "The value of a product increases as more people use it, deterring new competitors."},
#     {"Name": "Vertical Integration", "Definition": "Control over supply chain that limits competitors' access to resources or distribution."}
# ]

# # 轉換為訓練格式，並將 'answer' 設置為 'Name'
# train_data = [{"question": f"What is {item['Name']}?", "context": item["Definition"], "answer": item["Name"]} for item in data]

# # 分割數據集：80% 用於訓練，20% 用於驗證
# train_data_split, val_data_split = train_test_split(train_data, test_size=0.2, random_state=42)

# # 轉換為 Hugging Face Dataset 格式
# train_dataset = Dataset.from_dict({
#     "question": [entry["question"] for entry in train_data_split],
#     "context": [entry["context"] for entry in train_data_split],
#     "answer": [entry["answer"] for entry in train_data_split],  # 新增 answer 字段
# })

# val_dataset = Dataset.from_dict({
#     "question": [entry["question"] for entry in val_data_split],
#     "context": [entry["context"] for entry in val_data_split],
#     "answer": [entry["answer"] for entry in val_data_split],  # 新增 answer 字段
# })

# # 查看數據集格式
# print("Train Dataset:", train_dataset)
# print("Validation Dataset:", val_dataset)


# 合併DataFrame

In [24]:
import os
import pandas as pd

# 指定資料夾路徑
folder_path = r'C:\Users\User\Dropbox\textmining1\Data2_Opinion_low'  # 替換成您的資料夾路徑

# 遍歷資料夾中的所有 CSV 文件
csv_files = [f for f in os.listdir(folder_path) if f.endswith('_opinion.csv')]

# 用來儲存所有 CSV 的列表
dataframes = []

# 讀取每個 CSV 文件並加載到 DataFrame 中
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)  # 讀取 CSV
    dataframes.append(df)  # 將 DataFrame 添加到列表中

# 如果需要，將所有 DataFrame 合併成一個大的 DataFrame
final_df = pd.concat(dataframes, ignore_index=True)


# 顯示合併後的 DataFrame
print(final_df)


       Case Index  Page                                          Paragraph
0               1    26  Opinion\n[*42] ORDER\nWOLF, D.J.\nFebruary 3, ...
1               1    27                                       Page 7 of 15
2               1    27  Beverages, Inc. ("MWMBI"), a trade association...
3               1    28                                       Page 8 of 15
4               1    28  The Magistrate Judge's Report and Recommendati...
...           ...   ...                                                ...
24740         100  1875  Accordingly, considering the Kansas state cour...
24741         100  1876                                      Page 11 of 12
24742         100  1876  Plaintiffs' Complaint asserts that defendants'...
24743         100  1877                                      Page 12 of 12
24744         100  1877  obtaining relief does not trigger K.S.A. 60-51...

[24745 rows x 3 columns]


# 導入必要的庫和設置

In [2]:
# 1. Import necessary libraries
import os
import pandas as pd
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
from tqdm.notebook import tqdm  # 用於顯示進度條

# 2. Load tokenizer
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')


In [26]:
import torch
print(torch.cuda.is_available())  # 如果返回 True，則說明您的環境可以使用 GPU
print(torch.cuda.current_device())  # 顯示當前的 GPU 設備索引
print(torch.cuda.get_device_name(0))  # 顯示當前 GPU 的名稱


True
0
NVIDIA GeForce RTX 3060


In [ ]:
# 加載模型並將其移動到 GPU (如果有)
model = BertForQuestionAnswering.from_pretrained('nlpaueb/legal-bert-base-uncased')
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

# 讀取所有 CSV 文件並合併

In [3]:
# 3. Set folder path and read CSV files
folder_path = r'C:\Users\User\Dropbox\textmining1\Data2_Opinion_low'  # 替換成您的資料夾路徑

# Get all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('_opinion.csv')]

# Initialize list to store dataframes
dataframes = []

# Read each CSV file and append it to the list
for csv_file in tqdm(csv_files, desc="Reading CSV files"):
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)  # Read CSV
    dataframes.append(df)  # Add DataFrame to the list

# Merge all dataframes into one large dataframe
final_df = pd.concat(dataframes, ignore_index=True)

# Display merged dataframe
print(final_df.head())


Reading CSV files:   0%|          | 0/10 [00:00<?, ?it/s]

   Case Index  Page                                          Paragraph
0           1    26  Opinion\n[*42] ORDER\nWOLF, D.J.\nFebruary 3, ...
1           1    27                                       Page 7 of 15
2           1    27  Beverages, Inc. ("MWMBI"), a trade association...
3           1    28                                       Page 8 of 15
4           1    28  The Magistrate Judge's Report and Recommendati...


# 裁剪文本並創建問答對

In [29]:
# 4. Function to split text into chunks (max_length 512 tokens)
def split_text_into_chunks(text, max_length=256):
    # Tokenize the text, do not truncate yet
    tokens = tokenizer.encode(text, truncation=False)
    
    chunks = []
    
    # Split the tokens into chunks of size max_length
    while len(tokens) > max_length:
        chunk = tokens[:max_length]  # First chunk of max_length tokens
        chunks.append(chunk)         # Append the chunk to the list
        tokens = tokens[max_length:] # Remaining tokens to be processed
    
    # If there are leftover tokens, process them
    if len(tokens) > 0:
        chunks.append(tokens)
    
    # Ensure that each chunk is strictly max_length or less
    chunks = [chunk[:max_length] for chunk in chunks]  # Ensure no chunk exceeds max_length
    
    return chunks


In [30]:
# 5. Create Q&A pairs and split the text
train_data = []
for _, row in tqdm(final_df.iterrows(), total=final_df.shape[0], desc="Processing text"):
    case_index = row['Case Index']
    paragraph = row['Paragraph']
    
    # Split paragraph into chunks
    paragraph_chunks = split_text_into_chunks(paragraph)
    
    for chunk in paragraph_chunks:
        # Decode tokens back to text
        context = tokenizer.decode(chunk, skip_special_tokens=True)
        
        # Create question and answer pair
        question = f"What is the judge's opinion regarding entry barriers in case {case_index}?"
        train_data.append({"question": question, "context": context, "answer": context})
        
        # If prosecutor's argument is involved
        question_prosecutor = f"What is the prosecutor's argument regarding entry barriers in case {case_index}?"
        train_data.append({"question": question_prosecutor, "context": context, "answer": context})


Processing text:   0%|          | 0/24745 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (854 > 512). Running this sequence through the model will result in indexing errors


In [31]:
# Display a sample of the generated Q&A pairs
print(train_data[:50])  # Show first 5 entries


[{'question': "What is the judge's opinion regarding entry barriers in case 1?", 'context': 'opinion [ * 42 ] order wolf, d. j. february 3, 1998 the following analysis is based upon the transcript of the decision rendered orally on january 27, 1998, granting plaintiffs\'motions for summary judgment ( docket nos. 158 & 166 ) and denying defendants\'motions for summary judgment ( docket nos. 162 & 169 ). this memorandum adds citations, revises and amplifies some [ * * 2 ] of the discussion, and deletes certain non - essential matters. the transcripts of the hearings on january 26 and 27, 1998, are being prepared and may be acquired from the court reporter. * * * * the present plaintiffs in this case are sea shore corporation, which does business as canterbury liquors and pantry, a licensed retailer of alcoholic beverages, 1 and an intervenor as plaintiff, whitehall company limited, a licensed wholesaler of alcoholic [ * 43 ] beverages 2 [ * * 3 ] ( collectively, the " plaintiffs " ). the

# 檢查裁切文本長度

## 檢查段落的長度

In [32]:
# 假設您已經有一個處理過的 DataFrame `final_df` 和 `tokenizer`
def check_token_lengths(final_df, tokenizer, max_length=512):
    for _, row in df.iterrows():
        paragraph = row['Paragraph']
        
        # Tokenize the paragraph to check the length
        inputs = tokenizer(paragraph, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        
        # Print the length of tokens after tokenization
        print(f"Original length: {len(paragraph)} characters")
        print(f"Tokenized length: {len(inputs['input_ids'][0])} tokens")
        print(f"Tokenized text (first 50 tokens): {inputs['input_ids'][0][:50]}")  # Display first 50 tokens
        
# 執行檢查
check_token_lengths(final_df, tokenizer)


Original length: 784 characters
Tokenized length: 512 tokens
Tokenized text (first 50 tokens): tensor([  101,   528,   595,  1314,  3688,   184,   365,   469,  6047,   588,
          380,   403,  2486,   604,   243,   210, 10958,   178,   115,  3587,
          111,   380,   403,   112,   212,  1285,   569,   239,  2043,   211,
         6410,   145,   388,   217,   604,   771,   565,   359,   854,   211,
          145,   993,  5471,   213,   229,   595,   545,   145,   423,   126])
Original length: 12 characters
Tokenized length: 512 tokens
Tokenized text (first 50 tokens): tensor([101, 898, 199, 210, 337, 102,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0])
Original length: 4639 characters
Tokenized length: 512 tokens
Tokenized text (first 50 tokens): tensor([  101,   606,   595,   212,   380

## 確保文本正確裁剪

In [33]:
def split_text_into_chunks(text, max_length=256):
    tokens = tokenizer.encode(text, truncation=False)  # 不進行裁剪，先取得所有tokens
    print(f"Total tokens before splitting: {len(tokens)}")
    chunks = []
    
    # If the text exceeds max_length, split it
    while len(tokens) > max_length:
        chunk = tokens[:max_length]
        chunks.append(chunk)
        tokens = tokens[max_length:]
    
    # Add any remaining tokens
    if len(tokens) > 0:
        chunks.append(tokens)
    
    print(f"Number of chunks after splitting: {len(chunks)}")
    print(f"Tokens in the first chunk: {len(chunks[0])}")
    return chunks

# Example of splitting a long paragraph
example_paragraph = final_df.iloc[0]['Paragraph']  # Example paragraph from the dataset
split_chunks = split_text_into_chunks(example_paragraph)


Total tokens before splitting: 269
Number of chunks after splitting: 2
Tokens in the first chunk: 256


## 驗證裁剪後的文本內容

In [34]:
def check_split_chunks(chunks, tokenizer):
    for i, chunk in enumerate(chunks):
        text = tokenizer.decode(chunk, skip_special_tokens=True)  # Decode the chunk back to text
        print(f"Chunk {i+1}: {text[:100]}...")  # Display first 100 characters of each chunk

# Check chunks for a specific paragraph
check_split_chunks(split_chunks, tokenizer)


Chunk 1: opinion [ * 42 ] order wolf, d. j. february 3, 1998 the following analysis is based upon the transcr...
Chunk 2: . s. dist. lexis 1155, * * 1...


# 分割訓練集和測試集

In [35]:
# 6. Split the data into train and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "question": [entry["question"] for entry in train_data],
    "context": [entry["context"] for entry in train_data],
    "answer": [entry["answer"] for entry in train_data],
})

val_dataset = Dataset.from_dict({
    "question": [entry["question"] for entry in val_data],
    "context": [entry["context"] for entry in val_data],
    "answer": [entry["answer"] for entry in val_data],
})


In [36]:
# Check final dataset
print(train_dataset)
print(val_dataset)


Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 99068
})
Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 24768
})


In [37]:
from transformers import BertTokenizer  # 確保導入BertTokenizer
import torch
from datasets import Dataset

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

# Function to split text into chunks (max_length 512 tokens)
def split_text_into_chunks(text, max_length=256):
    tokens = tokenizer.encode(text, truncation=False)  # Don't truncate yet, just get all tokens
    chunks = []
    
    # Split the tokens into chunks of size max_length
    while len(tokens) > max_length:
        chunk = tokens[:max_length]
        chunks.append(chunk)
        tokens = tokens[max_length:]
    
    # Add any remaining tokens
    if len(tokens) > 0:
        chunks.append(tokens)
    
    print(f"Total tokens before splitting: {len(tokens)}")
    print(f"Number of chunks after splitting: {len(chunks)}")
    print(f"Tokens in the first chunk: {len(chunks[0])}")
    return chunks

# Example usage: Process a specific paragraph from the dataframe
example_paragraph = "This is an example paragraph. It will be tokenized and split into chunks."
split_chunks = split_text_into_chunks(example_paragraph)


Total tokens before splitting: 20
Number of chunks after splitting: 1
Tokens in the first chunk: 20


# 進行tokenization -> 生成 start/end positions -> 將數據移到GPU

In [38]:
from transformers import BertTokenizer
import torch

# Load tokenizer once
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

# Use device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [39]:
def tokenize_and_find_answers(examples, tokenizer):
    # Tokenizer 需要傳遞的資料
    questions = examples['question']
    contexts = examples['context']
    
    # 用 tokenizer 處理問題和上下文
    inputs = tokenizer(questions, contexts, truncation=True, padding='max_length', max_length=512, return_tensors='pt')

    # 處理每個上下文中的範例
    start_positions = []
    end_positions = []
    
    for i, context in enumerate(examples['context']):
        answer = examples['answer'][i]
        
        # 找到答案在上下文中的位置
        start_position = context.find(answer)
        end_position = start_position + len(answer)

        # 調整位置來符合 tokenized 輸入
        start_token_pos = len(tokenizer.encode(context[:start_position], truncation=True, padding='max_length', max_length=512)) - 1
        end_token_pos = len(tokenizer.encode(context[:end_position], truncation=True, padding='max_length', max_length=512)) - 1

        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)

    # 使用 torch.tensor 將位置轉為 PyTorch 張量
    inputs['start_positions'] = torch.tensor(start_positions)
    inputs['end_positions'] = torch.tensor(end_positions)

    return inputs


In [40]:
from transformers import BertTokenizer
import torch

# Load tokenizer once
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

# Use device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def tokenize_and_find_answers(examples, tokenizer):
    # Extract questions and contexts from the batch (which are lists)
    questions = examples['question']
    contexts = examples['context']
    
    # Tokenize the batch of questions and contexts
    inputs = tokenizer(questions, contexts, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    
    # Initialize lists to store start and end positions
    start_positions = []
    end_positions = []
    
    # For each example in the batch, calculate the start and end positions of the answer
    for i in range(len(contexts)):
        context = contexts[i]
        answer = examples['answer'][i]
        
        # Find the start and end position of the answer within the context
        start_position = context.find(answer)
        end_position = start_position + len(answer)
        
        # Adjust positions for the tokenized input
        start_token_pos = len(tokenizer.encode(context[:start_position], truncation=True, padding='max_length', max_length=512)) - 1
        end_token_pos = len(tokenizer.encode(context[:end_position], truncation=True, padding='max_length', max_length=512)) - 1
        
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
    
    # Convert the positions to tensors and add them to the inputs
    inputs['start_positions'] = torch.tensor(start_positions).to(device)
    inputs['end_positions'] = torch.tensor(end_positions).to(device)

    return inputs

# 進行批次處理，這次確保處理的是批次資料
train_dataset = train_dataset.map(
    tokenize_and_find_answers, 
    remove_columns=["question", "context", "answer"], 
    num_proc=1,  # Parallel processing
    batched=True,  # Process in batches
    fn_kwargs={'tokenizer': tokenizer}  # Pass tokenizer explicitly
)

val_dataset = val_dataset.map(
    tokenize_and_find_answers, 
    remove_columns=["question", "context", "answer"], 
    num_proc=1,  # Parallel processing
    batched=True,  # Process in batches
    fn_kwargs={'tokenizer': tokenizer}  # Pass tokenizer explicitly
)


Map:   0%|          | 0/99068 [00:00<?, ? examples/s]

Map:   0%|          | 0/24768 [00:00<?, ? examples/s]

## 確認訓練集和測試集

In [41]:
# Display the final processed dataset
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 99068
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 24768
})


# 確認GPU

In [48]:
print(f"Using device: {device}")

Using device: cuda


## 設置Trainer並開始訓練

## 清除之前train的緩存

In [51]:
import shutil
shutil.rmtree("./results", ignore_errors=True)  # 刪除保存路徑的內容

shutil.rmtree("./logs", ignore_errors=True)  # 清除舊日誌


### Define training arguments

In [ ]:
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments

# 載入模型
model = BertForQuestionAnswering.from_pretrained('nlpaueb/legal-bert-base-uncased')

# 設置訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# 設置 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
 
# 開始訓練
trainer.train()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\User\Dropbox\textmining1\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/18576 [00:00<?, ?it/s]

c:\Users\User\Dropbox\textmining1\.venv\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


{'loss': 1.1837, 'grad_norm': 8.810790061950684, 'learning_rate': 1.9468130921619294e-05, 'epoch': 0.08}


c:\Users\User\Dropbox\textmining1\.venv\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

{'loss': 0.0, 'grad_norm': 0.00015015251119621098, 'learning_rate': 9.259259259259259e-08, 'epoch': 2.99}


c:\Users\User\Dropbox\textmining1\.venv\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


  0%|          | 0/1548 [00:00<?, ?it/s]

{'eval_loss': 1.066227127921593e-06, 'eval_runtime': 164.0956, 'eval_samples_per_second': 150.936, 'eval_steps_per_second': 9.434, 'epoch': 3.0}
{'train_runtime': 7389.1059, 'train_samples_per_second': 40.222, 'train_steps_per_second': 2.514, 'train_loss': 0.06685295676333312, 'epoch': 3.0}


TrainOutput(global_step=18576, training_loss=0.06685295676333312, metrics={'train_runtime': 7389.1059, 'train_samples_per_second': 40.222, 'train_steps_per_second': 2.514, 'total_flos': 7.765844128896614e+16, 'train_loss': 0.06685295676333312, 'epoch': 3.0})

### evaluate the model after training

In [54]:
trainer.evaluate()

  0%|          | 0/1548 [00:00<?, ?it/s]

{'eval_loss': 1.066227127921593e-06,
 'eval_runtime': 166.0984,
 'eval_samples_per_second': 149.116,
 'eval_steps_per_second': 9.32,
 'epoch': 3.0}

In [56]:
# 保存微調後的模型
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.txt',
 './fine_tuned_model\\added_tokens.json')

# 進行推理

## 加載新的Validation Data進行測試並切割判例段落防止Token length太長

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置文件夾路徑
folder_path = 'Data2_Opinion_valid'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取文件夾中的 CSV 文件
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 遍歷 CSV 文件並顯示進度條
for csv_file in tqdm(csv_files, desc="Processing CSV Files"):
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    
    # 遍歷 DataFrame 中的每行並顯示進度條
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}", leave=False):
        case_index = row['Case Index']  # 替換成您的列名
        paragraph = row['Paragraph']    # 替換成您的列名
        
        # 對段落進行處理和滑動窗口切割
        chunks = preprocess_and_split_with_overlap(paragraph)
        
        for i, chunk in enumerate(chunks):
            processed_data.append({
                'CSV File': csv_file,  # 添加 CSV 文件名稱
                'Case Index': case_index,
                'Chunk ID': i,
                'Chunk Text': chunk
            })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_Opinion.csv', index=False)

print("處理完成，已保存為 'Processed_Opinion.csv'")


Processing CSV Files: 100%|██████████| 30/30 [20:38<00:00, 41.27s/it]


處理完成，已保存為 'Processed_Opinion.csv'


## 加載模型、知識圖譜, 定義檢索知識圖譜函數、RAG測試Pipeline、預測函數

In [4]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open("definition.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數

def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = []
    for entry in knowledge_graph:
        if entry["Name"].lower() in answer.lower():
            related_definitions.append(entry)
    
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線

def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    csv_file = row['CSV File']  # 新增：來源文件名稱
    case_index = row['Case Index']
    paragraph = row['Chunk Text']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case {case_index}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, paragraph, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "CSV File": csv_file,  # 新增：來源文件名稱
        "Case Index": case_index,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數

def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        # 編碼輸入
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # 移動到 GPU
        
        # 模型推理
        with torch.no_grad():
            outputs = model(**inputs)
        
        # 提取答案位置
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions


Loaded knowledge graph with 13 entries.


## 加載處理過的驗證數據並執行測試

In [5]:
# 加載處理後的數據
valid_data = pd.read_csv("Processed_Opinion.csv")

# 提取問題和上下文，並加入文件名
questions = [f"What is the judge's opinion regarding entry barriers in case {case}?" 
             for case in valid_data["Case Index"]]
contexts = valid_data["Chunk Text"].tolist()
csv_files = valid_data["CSV File"].tolist()  # 使用 'CSV File' 列

print(f"Loaded {len(questions)} samples for validation.")

# 移動模型到 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)

## 知識圖譜檢索
def generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 Case Index 和 CSV 文件名的結果列表。
    """
    results = []
    for case_index, csv_file, question, context, prediction in zip(valid_data["Case Index"], csv_files, questions, contexts, predicted_answers):
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "Case Index": case_index,
            "CSV File": csv_file,  # 包含 CSV 文件名
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph)



Loaded 275371 samples for validation.


Predicting: 100%|██████████| 17211/17211 [49:19<00:00,  5.82it/s]


Results saved to 'RAG_Test_Results_With_CSV_File.json'


In [ ]:
# 將結果保存為 JSON 文件
import json
with open("RAG_Test_Results_With_CSV_File.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results_With_CSV_File.json'")


## 查看測試結果

In [7]:
# 查看部分結果
for result in results[:5]:
    print("CSV File", result["CSV File"])
    print("Case Index:", result["Case Index"])  # 新增 Case Index 的輸出
    print("Question:", result["Question"])
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


CSV File 1015 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 1015 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 1015 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 1015 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 1015 (1)_opinion.csv
Case Index: 1
Question: What i

## 查看Match的數據

In [8]:
# 載入保存的結果
import json

with open("RAG_Test_Results_With_CSV_File.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("CSV File", result["CSV File"])   
    print(f"Case Index: {result['Case Index']}")  # 包含 Case Index
    print(f"Question: {result['Question']}")
    print(f"Context: {result['Context'][:500]}...")
    print(f"Predicted Answer: {result['Predicted Answer']}")
    print(f"Knowledge Graph Result: {result['Knowledge Graph Result']}")
    print("=" * 50)


Found 17 matched results.
CSV File 1015 (10)_opinion.csv
Case Index: 4
Question: What is the judge's opinion regarding entry barriers in case 4?
Context: power is " the ability to... increase... profits by reducing output and charging more than a competitive price. " hovenkamp, supra, antitrust law § 3. 1. monopoly power is a high dosage of market power. id. § 5. 2. although various factors are important in determining whether a firm has monopoly power ( such as entry barriers, economies of scale, concentration ), often the analysis turns on market share - - generally, the higher the market share the easier is the plaintiff's case. id. usually, c...
Predicted Answer: [SEP] power is " the ability to . . . increase . . . profits by reducing output and charging more than a competitive price . " hovenkamp , supra , antitrust law § 3 . 1 . monopoly power is a high dosage of market power . id . § 5 . 2 . although various factors are important in determining whether a firm has monopoly power 

In [10]:
# 將匹配的結果保存為新的 JSON 文件
output_file = "RAGG_Test_Results_With_CSV_File.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(matched_results, f, ensure_ascii=False, indent=4)

print(f"Matched results saved to '{output_file}'")


Matched results saved to 'RAGG_Test_Results_With_CSV_File.json'


# Data1

In [14]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置文件夾路徑
folder_path = 'Data1_Opinion'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取文件夾中的 CSV 文件
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 遍歷 CSV 文件並顯示進度條
for csv_file in tqdm(csv_files, desc="Processing CSV Files"):
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    
    # 遍歷 DataFrame 中的每行並顯示進度條
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}", leave=False):
        case_index = row['Case Index']  # 替換成您的列名
        paragraph = row['Paragraph']    # 替換成您的列名
        
        # 對段落進行處理和滑動窗口切割
        chunks = preprocess_and_split_with_overlap(paragraph)
        
        for i, chunk in enumerate(chunks):
            processed_data.append({
                'CSV File': csv_file,  # 添加 CSV 文件名稱
                'Case Index': case_index,
                'Chunk ID': i,
                'Chunk Text': chunk
            })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_Opinion_Data1.csv', index=False)

print("處理完成，已保存為 'Processed_Opinion_Data1.csv'")



Processing CSV Files: 100%|██████████| 30/30 [17:23<00:00, 34.77s/it]


處理完成，已保存為 'Processed_Opinion_Data1.csv'


In [16]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open("definition.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數

def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = []
    for entry in knowledge_graph:
        if entry["Name"].lower() in answer.lower():
            related_definitions.append(entry)
    
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線

def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    csv_file = row['CSV File']  # 新增：來源文件名稱
    case_index = row['Case Index']
    paragraph = row['Chunk Text']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case {case_index}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, paragraph, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "CSV File": csv_file,  # 新增：來源文件名稱
        "Case Index": case_index,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數

def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        # 編碼輸入
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # 移動到 GPU
        
        # 模型推理
        with torch.no_grad():
            outputs = model(**inputs)
        
        # 提取答案位置
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions

# 加載處理後的數據
valid_data = pd.read_csv("Processed_Opinion_Data1.csv")

# 提取問題和上下文，並加入文件名
questions = [f"What is the judge's opinion regarding entry barriers in case {case}?" 
             for case in valid_data["Case Index"]]
contexts = valid_data["Chunk Text"].tolist()
csv_files = valid_data["CSV File"].tolist()  # 使用 'CSV File' 列

print(f"Loaded {len(questions)} samples for validation.")

# 移動模型到 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)

## 知識圖譜檢索
def generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 Case Index 和 CSV 文件名的結果列表。
    """
    results = []
    for case_index, csv_file, question, context, prediction in zip(valid_data["Case Index"], csv_files, questions, contexts, predicted_answers):
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "Case Index": case_index,
            "CSV File": csv_file,  # 包含 CSV 文件名
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph)


Loaded knowledge graph with 13 entries.
Loaded 234718 samples for validation.


Predicting: 100%|██████████| 14670/14670 [42:23<00:00,  5.77it/s]


In [17]:
# 將結果保存為 JSON 文件
import json
with open("RAG_Test_Results_With_CSV_File_Data1.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results_With_CSV_File_Data1.json'")

# 查看部分結果
for result in results[:5]:
    print("CSV File", result["CSV File"])
    print("Case Index:", result["Case Index"])  # 新增 Case Index 的輸出
    print("Question:", result["Question"])
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


Results saved to 'RAG_Test_Results_With_CSV_File_Data1.json'
CSV File 0101 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 0101 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 0101 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 0101 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]

In [ ]:
# 載入保存的結果
import json

with open("RAG_Test_Results_With_CSV_File_Data1.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("CSV File", result["CSV File"])   
    print(f"Case Index: {result['Case Index']}")  # 包含 Case Index
    print(f"Question: {result['Question']}")
    print(f"Context: {result['Context'][:500]}...")
    print(f"Predicted Answer: {result['Predicted Answer']}")
    print(f"Knowledge Graph Result: {result['Knowledge Graph Result']}")
    print("=" * 50)


Found 16 matched results.
CSV File 0101 (1)_opinion.csv
Case Index: 92
Question: What is the judge's opinion regarding entry barriers in case 92?
Context: monopoly does not violate the antitrust laws unless it was acquired or maintained by exclusionary, unfair, or predatory means. " 29 the trial judge further instructed the jury, however, that hecht bore the burden of proving that the redskins did not have a natural monopoly : 30 natural monopoly. in the economic sense, natural monopoly is monopoly resulting from economies of scale, a relationship between the size of the market and the size of the most efficient firm such that one firm of efficien...
Predicted Answer: [SEP] monopoly does not violate the antitrust laws unless it was acquired or maintained by exclusionary , unfair , or predatory means . " 29 the trial judge further instructed the jury , however , that hecht bore the burden of proving that the redskins did not have a natural monopoly : 30 natural monopoly . in the economi

# Data3

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置文件夾路徑
folder_path = 'Data3_Opinion'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取文件夾中的 CSV 文件
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 遍歷 CSV 文件並顯示進度條
for csv_file in tqdm(csv_files, desc="Processing CSV Files"):
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    
    # 遍歷 DataFrame 中的每行並顯示進度條
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}", leave=False):
        case_index = row['Case Index']  # 替換成您的列名
        paragraph = row['Paragraph']    # 替換成您的列名
        
        # 對段落進行處理和滑動窗口切割
        chunks = preprocess_and_split_with_overlap(paragraph)
        
        for i, chunk in enumerate(chunks):
            processed_data.append({
                'CSV File': csv_file,  # 添加 CSV 文件名稱
                'Case Index': case_index,
                'Chunk ID': i,
                'Chunk Text': chunk
            })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_Opinion_Data3.csv', index=False)

print("處理完成，已保存為 'Processed_Opinion_Data3.csv'")


Processing CSV Files: 100%|██████████| 34/34 [26:30<00:00, 46.79s/it]


處理完成，已保存為 'Processed_Opinion_Data3.csv'


In [21]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open("definition.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數

def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = []
    for entry in knowledge_graph:
        if entry["Name"].lower() in answer.lower():
            related_definitions.append(entry)
    
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線

def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    csv_file = row['CSV File']  # 新增：來源文件名稱
    case_index = row['Case Index']
    paragraph = row['Chunk Text']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case {case_index}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, paragraph, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "CSV File": csv_file,  # 新增：來源文件名稱
        "Case Index": case_index,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數

def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        # 編碼輸入
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # 移動到 GPU
        
        # 模型推理
        with torch.no_grad():
            outputs = model(**inputs)
        
        # 提取答案位置
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions

# 加載處理後的數據
valid_data = pd.read_csv("Processed_Opinion_Data3.csv")

# 提取問題和上下文，並加入文件名
questions = [f"What is the judge's opinion regarding entry barriers in case {case}?" 
             for case in valid_data["Case Index"]]
contexts = valid_data["Chunk Text"].tolist()
csv_files = valid_data["CSV File"].tolist()  # 使用 'CSV File' 列

print(f"Loaded {len(questions)} samples for validation.")

# 移動模型到 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)

## 知識圖譜檢索
def generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 Case Index 和 CSV 文件名的結果列表。
    """
    results = []
    for case_index, csv_file, question, context, prediction in zip(valid_data["Case Index"], csv_files, questions, contexts, predicted_answers):
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "Case Index": case_index,
            "CSV File": csv_file,  # 包含 CSV 文件名
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph)



Loaded knowledge graph with 13 entries.
Loaded 360699 samples for validation.


Predicting: 100%|██████████| 22544/22544 [1:04:11<00:00,  5.85it/s]


In [22]:
import json
with open("RAG_Test_Results_With_CSV_File_Data3.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results_With_CSV_File_Data3.json'")

# 查看部分結果
for result in results[:5]:
    print("CSV File", result["CSV File"])
    print("Case Index:", result["Case Index"])  # 新增 Case Index 的輸出
    print("Question:", result["Question"])
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


Results saved to 'RAG_Test_Results_With_CSV_File_Data3.json'
CSV File 0423 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 0423 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 0423 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: ,
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File 0423 (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV

In [23]:
# 載入保存的結果
import json

with open("RAG_Test_Results_With_CSV_File_Data3.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("CSV File", result["CSV File"])   
    print(f"Case Index: {result['Case Index']}")  # 包含 Case Index
    print(f"Question: {result['Question']}")
    print(f"Context: {result['Context'][:500]}...")
    print(f"Predicted Answer: {result['Predicted Answer']}")
    print(f"Knowledge Graph Result: {result['Knowledge Graph Result']}")
    print("=" * 50)


Found 7 matched results.
CSV File 0423 (4)_opinion.csv
Case Index: 93
Question: What is the judge's opinion regarding entry barriers in case 93?
Context: ##c distribution market in northern california. ( nsw's [ * 20 ] motion for summary judgment re antitrust claims at 9. ) plaintiffs respond that defendant nsw's exclusive contracts to distribute certain hvac products combined with evidence of flat spec'ing practices on hvac projects are sufficient to create a genuine issue of material fact as to whether there are significant barriers to entry. 30 as the ninth circuit has explained, a § 2 plaintiff, establishing monopoly power by circumstantial e...
Predicted Answer: [SEP] # # c distribution market in northern california . ( nsw ' s [ * 20 ] motion for summary judgment re antitrust claims at 9 . ) plaintiffs respond that defendant nsw ' s exclusive contracts to distribute certain hvac products combined with evidence of flat spec ' ing practices on hvac projects are sufficient to create

# Supreme Court

In [8]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置文件夾路徑
folder_path = 'Opinion_Supreme'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取文件夾中的 CSV 文件
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 遍歷 CSV 文件並顯示進度條
for csv_file in tqdm(csv_files, desc="Processing CSV Files"):
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    
    # 遍歷 DataFrame 中的每行並顯示進度條
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}", leave=False):
        case_index = row['Case Index']  # 替換成您的列名
        paragraph = row['Paragraph']    # 替換成您的列名
        
        # 對段落進行處理和滑動窗口切割
        chunks = preprocess_and_split_with_overlap(paragraph)
        
        for i, chunk in enumerate(chunks):
            processed_data.append({
                'CSV File': csv_file,  # 添加 CSV 文件名稱
                'Case Index': case_index,
                'Chunk ID': i,
                'Chunk Text': chunk
            })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_Opinion_Supreme.csv', index=False)

print("處理完成，已保存為 'Processed_Opinion_Supreme.csv'")


Processing CSV Files: 100%|██████████| 19/19 [00:14<00:00,  1.28it/s]


處理完成，已保存為 'Processed_Opinion_Supreme.csv'


In [9]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open("definition.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數

def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = []
    for entry in knowledge_graph:
        if entry["Name"].lower() in answer.lower():
            related_definitions.append(entry)
    
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線

def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    csv_file = row['CSV File']  # 新增：來源文件名稱
    case_index = row['Case Index']
    paragraph = row['Chunk Text']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case {case_index}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, paragraph, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "CSV File": csv_file,  # 新增：來源文件名稱
        "Case Index": case_index,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數

def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        # 編碼輸入
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # 移動到 GPU
        
        # 模型推理
        with torch.no_grad():
            outputs = model(**inputs)
        
        # 提取答案位置
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions

# 加載處理後的數據
valid_data = pd.read_csv("Processed_Opinion_Supreme.csv")

# 提取問題和上下文，並加入文件名
questions = [f"What is the judge's opinion regarding entry barriers in case {case}?" 
             for case in valid_data["Case Index"]]
contexts = valid_data["Chunk Text"].tolist()
csv_files = valid_data["CSV File"].tolist()  # 使用 'CSV File' 列

print(f"Loaded {len(questions)} samples for validation.")

# 移動模型到 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)

## 知識圖譜檢索
def generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 Case Index 和 CSV 文件名的結果列表。
    """
    results = []
    for case_index, csv_file, question, context, prediction in zip(valid_data["Case Index"], csv_files, questions, contexts, predicted_answers):
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "Case Index": case_index,
            "CSV File": csv_file,  # 包含 CSV 文件名
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results_with_csv(valid_data, questions, contexts, csv_files, predicted_answers, knowledge_graph)



Loaded knowledge graph with 13 entries.
Loaded 3269 samples for validation.


Predicting: 100%|██████████| 205/205 [00:35<00:00,  5.81it/s]


In [10]:
import json
with open("RAG_Test_Results_With_CSV_File_Supreme.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results_With_CSV_File_Supreme.json'")

# 查看部分結果
for result in results[:5]:
    print("CSV File", result["CSV File"])
    print("Case Index:", result["Case Index"])  # 新增 Case Index 的輸出
    print("Question:", result["Question"])
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


Results saved to 'RAG_Test_Results_With_CSV_File_Supreme.json'
CSV File Supreme (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File Supreme (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File Supreme (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
CSV File Supreme (1)_opinion.csv
Case Index: 1
Question: What is the judge's opinion regarding entry barriers in case 1?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related defini

In [12]:
# 載入保存的結果
import json

with open("RAG_Test_Results_With_CSV_File_Supreme.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("CSV File", result["CSV File"])   
    print(f"Case Index: {result['Case Index']}")  # 包含 Case Index
    print(f"Question: {result['Question']}")
    print(f"Context: {result['Context'][:500]}...")
    print(f"Predicted Answer: {result['Predicted Answer']}")
    print(f"Knowledge Graph Result: {result['Knowledge Graph Result']}")
    print("=" * 50)


Found 0 matched results.


# 涵蓋更多entryBarrier

In [24]:
import os
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open("definition_moreinfo.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數
def retrieve_from_knowledge_graph(answer, knowledge_graph):
    related_definitions = []
    for entry in knowledge_graph:
        # 獲取名稱和同義詞
        synonyms = entry.get("Synonyms", [])
        terms_to_match = [entry["Name"]] + synonyms

        # 名稱或同義詞匹配
        if any(term.lower() in answer.lower() for term in terms_to_match):
            related_definitions.append(entry)
            continue

        # 定義關鍵詞匹配
        if entry["Definition"].lower() in answer.lower():
            related_definitions.append(entry)

    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義批量處理函數，新增檢索結合
def batch_predict_with_retrieval(model, tokenizer, questions, contexts, knowledge_graph, batch_size=16):
    predictions = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = []
        batch_contexts = contexts[i:i + batch_size]

        # 檢索知識並組合問題
        for question, context in zip(questions[i:i + batch_size], batch_contexts):
            # 模擬初步回答以檢索知識
            retrieved_knowledge = retrieve_from_knowledge_graph(context, knowledge_graph)
            retrieved_text = " ".join([item["Definition"] for item in retrieved_knowledge if item["Name"] != "No Match"])
            full_question = f"{retrieved_text} {question}"
            batch_questions.append(full_question)

        # 編碼輸入
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # 移動到 GPU

        with torch.no_grad():
            outputs = model(**inputs)

        # 解碼回答
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    return predictions

# 遞迴處理所有處理過的 CSV 文件
folder_path = "Processed_Data_Opinion"
all_results = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for file in tqdm(os.listdir(folder_path), desc="Processing Files"):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        valid_data = pd.read_csv(file_path)

        questions = [f"What is the judge's opinion regarding entry barriers in case {case}?" 
                     for case in valid_data["Case Index"]]
        contexts = valid_data["Chunk Text"].tolist()
        csv_files = [file] * len(questions)  # 文件名對應每個樣本

        predicted_answers = batch_predict_with_retrieval(model, tokenizer, questions, contexts, knowledge_graph)

        for case_index, csv_file, question, context, prediction in zip(valid_data["Case Index"], csv_files, questions, contexts, predicted_answers):
            related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
            all_results.append({
                "Case Index": case_index,
                "CSV File": csv_file,
                "Question": question,
                "Context": context,
                "Predicted Answer": prediction,
                "Knowledge Graph Result": related_definitions
            })

# 保存所有結果
with open("All_RAG_Results.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

print("All results saved to 'All_RAG_Results.json'")


Loaded knowledge graph with 13 entries.


Processing Files: 100%|██████████| 4/4 [2:38:13<00:00, 2373.30s/it]


All results saved to 'All_RAG_Results.json'


In [25]:
import json

# 載入所有結果
with open("All_RAG_Results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if any(res["Name"] != "No Match" for res in result["Knowledge Graph Result"])
]

# 列出匹配的結果數量
print(f"Found {len(matched_results)} matched results.")

# 保存匹配結果到新文件
output_file = "Matched_ALL_DATA.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(matched_results, f, ensure_ascii=False, indent=4)

# 顯示部分匹配結果
if matched_results:
    print("\nDisplaying the first 5 matched results:")
    for result in matched_results[:5]:  # 只顯示前5個匹配結果
        print("=" * 50)
        print(f"CSV File: {result['CSV File']}")
        print(f"Case Index: {result['Case Index']}")
        print(f"Question: {result['Question']}")
        print(f"Context: {result['Context'][:300]}...")  # 限制輸出長度
        print(f"Predicted Answer: {result['Predicted Answer']}")
        print("Knowledge Graph Result:")
        for kg_result in result["Knowledge Graph Result"]:
            print(f"  - Name: {kg_result['Name']}")
            print(f"    Definition: {kg_result['Definition']}")
        print("=" * 50)
else:
    print("No matched results found.")

print(f"Matched results saved to '{output_file}'")


Found 61 matched results.

Displaying the first 5 matched results:
CSV File: Processed_Opinion_Data1.csv
Case Index: 25
Question: What is the judge's opinion regarding entry barriers in case 25?
Context: s is $ 6, 000, 000. appellees offered testimony that the merger would enable certain economies of scale, specifically, that it would enable the formation of a more elaborate foreign department than either bank is presently able to maintain. but this attempted justification, which was not mentioned b...
Predicted Answer: [SEP] s is $ 6 , 000 , 000 . appellees offered testimony that the merger would enable certain economies of scale , specifically , that it would enable the formation of a more elaborate foreign department than either bank is presently able to maintain . but this attempted justification , which was not mentioned by the district court in its opinion and has not been developed with any fullness before this court , we consider abandoned . the exclusion of banks from the ftc 

# FN, HN, CoreTerms

## FN

In [4]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置 CSV 文件路徑
csv_file = r'/Users/tangjiahong/Dropbox/textmining1/Data_Clean/CSV/merge_CSV/merge_FN.csv'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取 CSV 文件
df = pd.read_csv(csv_file)

# 遍歷 DataFrame 中的每行並顯示進度條
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}", leave=False):
    pdf_name = row['PDF']  # PDF 文件名稱
    page_number = row['Page']  # 頁碼
    label = row['Label']  # 標籤
    paragraph = row['Text']  # 需要處理的文本內容
    
    # 對段落進行處理和滑動窗口切割
    chunks = preprocess_and_split_with_overlap(paragraph)
    
    for i, chunk in enumerate(chunks):
        processed_data.append({
            'PDF': pdf_name,
            'Page': page_number,
            'Label': label,
            'Text': chunk
        })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_FN_Data.csv', index=False)

print("處理完成，已保存為 'Processed_FN_Data.csv'")


Processing /Users/tangjiahong/Dropbox/textmining1/Data_Clean/CSV/merge_CSV/merge_FN.csv:   0%|          | 212/129441 [00:00<03:06, 694.38it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (681 > 512). Running this sequence through the model will result in indexing errors


處理完成，已保存為 'Processed_FN_Data.csv'


In [5]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open(r"C:\Users\User\Dropbox\textmining1\KG\definition_moreinfo.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數
def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = [entry for entry in knowledge_graph if entry["Name"].lower() in answer.lower()]
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線
def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    pdf_file = row['PDF']  # PDF 文件名稱
    page_number = row['Page']  # 頁碼
    label = row['Label']  # 標籤
    paragraph = row['Text']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case from {pdf_file}, page {page_number}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, paragraph, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "PDF": pdf_file,
        "Page": page_number,
        "Label": label,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數
def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions

# 加載處理後的數據
valid_data = pd.read_csv("Processed_FN_Data.csv")

# **填補 NaN 值**
valid_data["Label"] = valid_data["Label"].fillna(method="ffill")  # 用前一個非 NaN 值填補 Label
valid_data["Text"] = valid_data["Text"].fillna(method="ffill")  # 用前一個非 NaN 值填補 Text

# **確保所有 Text 欄位都是字串**
valid_data["Text"] = valid_data["Text"].astype(str)

# 提取問題和上下文
questions = [
    f"What is the judge's opinion regarding entry barriers in case from {pdf}, page {page}?"
    for pdf, page in zip(valid_data["PDF"], valid_data["Page"])
]
contexts = valid_data["Text"].tolist()

print(f"Loaded {len(questions)} samples for validation.")

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)

# 生成結果
def generate_results(valid_data, questions, contexts, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 PDF 和頁碼的結果列表。
    """
    results = []
    for pdf, page, label, question, context, prediction in zip(
        valid_data["PDF"], valid_data["Page"], valid_data["Label"], questions, contexts, predicted_answers):
        
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "PDF": pdf,
            "Page": page,
            "Label": label,
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results(valid_data, questions, contexts, predicted_answers, knowledge_graph)


Loaded knowledge graph with 13 entries.


C:\Users\User\AppData\Local\Temp\ipykernel_19972\2680819668.py:99: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  valid_data["Label"] = valid_data["Label"].fillna(method="ffill")  # 用前一個非 NaN 值填補 Label
C:\Users\User\AppData\Local\Temp\ipykernel_19972\2680819668.py:100: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  valid_data["Text"] = valid_data["Text"].fillna(method="ffill")  # 用前一個非 NaN 值填補 Text


Loaded 145859 samples for validation.


Predicting: 100%|██████████| 9117/9117 [22:06<00:00,  6.88it/s]


In [6]:
# 將結果保存為 JSON 文件
import json
# 將結果保存為 JSON 文件
with open("RAG_Test_Results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results.json'")

# 查看部分結果
for result in results[:5]:
    print("PDF:", result["PDF"])
    print("Page:", result["Page"])
    print("Label:", result["Label"])
    print("Question:", result["Question"])
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


Results saved to 'RAG_Test_Results.json'
PDF: a1
Page: 33
Label: 1
Question: What is the judge's opinion regarding entry barriers in case from a1, page 33?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1
Page: 33
Label: 2
Question: What is the judge's opinion regarding entry barriers in case from a1, page 33?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1
Page: 34
Label: 2
Question: What is the judge's opinion regarding entry barriers in case from a1, page 34?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1
Page: 35
Label: 3
Question: What is the judge's opinion regarding entry barriers in case from a1, page 35?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1
Page: 36
Label: 4
Quest

In [13]:
# 載入保存的結果
import json

# 加載結果
with open("RAG_FN_Results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("PDF:", result["PDF"])
    print("Page:", result["Page"])
    print("Label:", result["Label"])
    print("Question:", result["Question"])
    print("Context:", result["Context"][:500], "...")
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)

# 列出匹配的結果數量
print(f"Found {len(matched_results)} matched results.")

# 保存匹配結果到新文件
output_file = "Matched_FN_DATA.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(matched_results, f, ensure_ascii=False, indent=4)


Found 21 matched results.
PDF: b1
Page: 233
Label: 38
Question: What is the judge's opinion regarding entry barriers in case from b1, page 233?
Context: and, a fortiori, as the district court concluded, spcc has failed to prove that at & t engaged in predatory pricing under the areeda - turner test. ...
Predicted Answer: [SEP] and , a fortiori , as the district court concluded , spcc has failed to prove that at & t engaged in predatory pricing under the areeda - turner test . [SEP]
Knowledge Graph Result: [{'Name': 'Predatory Pricing', 'Definition': 'Deliberate underpricing to drive out competitors or prevent new entry.'}]
PDF: b5
Page: 979
Label: 24
Question: What is the judge's opinion regarding entry barriers in case from b5, page 979?
Context: . the court has also discussed whether there was below marginal cost pricing of any product, aside from the question of whether that product had sufficient market importance to even support a claim of predatory pricing, and aside from the que

## HN

In [7]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置 CSV 文件路徑
csv_file = r'/Users/tangjiahong/Dropbox/textmining1/Data_Clean/CSV/merge_CSV/merged_HN.csv'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取 CSV 文件
df = pd.read_csv(csv_file)

# 遍歷 DataFrame 中的每行並顯示進度條
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}", leave=False):
    pdf_name = row['PDF']  # PDF 文件名稱
    category = row['Category']  # 類別
    content = row['Content']  # 需要處理的文本內容
    
    # 對內容進行處理和滑動窗口切割
    chunks = preprocess_and_split_with_overlap(content)
    
    for i, chunk in enumerate(chunks):
        processed_data.append({
            'PDF': pdf_name,
            'Category': category,
            'Content': chunk
        })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_HN_Data.csv', index=False)

print("處理完成，已保存為 'Processed_HN_Data.csv'")

處理完成，已保存為 'Processed_HN_Data.csv'


In [10]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open(r"C:\Users\User\Dropbox\textmining1\KG\definition_moreinfo.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數
def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = [entry for entry in knowledge_graph if entry["Name"].lower() in answer.lower()]
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線
def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    pdf_file = row['PDF']  # PDF 文件名稱
    category = row['Category']  # 類別
    content = row['Content']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case from {pdf_file} in category {category}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, content, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "PDF": pdf_file,
        "Category": category,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數
def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions

# 加載處理後的數據
valid_data = pd.read_csv("Processed_HN_Data.csv")

# 提取問題和上下文
questions = [f"What is the judge's opinion regarding entry barriers in case from {pdf} in category {category}?"
             for pdf, category in zip(valid_data["PDF"], valid_data["Category"])]
contexts = valid_data["Content"].tolist()

print(f"Loaded {len(questions)} samples for validation.")

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)

# 生成結果
def generate_results(valid_data, questions, contexts, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 PDF 和類別的結果列表。
    """
    results = []
    for pdf, category, question, context, prediction in zip(
        valid_data["PDF"], valid_data["Category"], questions, contexts, predicted_answers):
        
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "PDF": pdf,
            "Category": category,
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results(valid_data, questions, contexts, predicted_answers, knowledge_graph)

Loaded knowledge graph with 13 entries.
Loaded 254703 samples for validation.


Predicting: 100%|██████████| 15919/15919 [08:42<00:00, 30.45it/s]


In [12]:
# 將結果保存為 JSON 文件
import json

# 將結果保存為 JSON 文件
with open("RAG_Test_Results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results.json'")

# 加載結果
with open("RAG_Test_Results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("PDF:", result["PDF"])
    print("Category:", result["Category"])
    print("Question:", result["Question"])
    print("Context:", result["Context"][:500], "...")
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


# 保存匹配結果到新文件
output_file = "Matched_HN_DATA.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(matched_results, f, ensure_ascii=False, indent=4)


Results saved to 'RAG_Test_Results.json'
Found 21 matched results.
PDF: c1
Category: HN10
Question: What is the judge's opinion regarding entry barriers in case from c1 in category HN10?
Context: antitrust & trade law >... > actual monopolization > anticompetitive & predatory practices > predatory pricing ...
Predicted Answer: [SEP] antitrust & trade law > . . . > actual monopolization > anticompetitive & predatory practices > predatory pricing [SEP]
Knowledge Graph Result: [{'Name': 'Predatory Pricing', 'Definition': 'Deliberate underpricing to drive out competitors or prevent new entry.'}]
PDF: c6
Category: Headnotes
Question: What is the judge's opinion regarding entry barriers in case from c6 in category Headnotes?
Context: predatory pricing - - > headnote : ...
Predicted Answer: [SEP] predatory pricing - - > headnote : [SEP]
Knowledge Graph Result: [{'Name': 'Predatory Pricing', 'Definition': 'Deliberate underpricing to drive out competitors or prevent new entry.'}]
PDF: c6
Catego

## CoreTerms

In [23]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer

# 設置 CSV 文件路徑
csv_file = r'/Users/tangjiahong/Dropbox/textmining1/Data_Clean/CSV/merge_CSV/merge_CoreTerms.csv'

# 初始化分詞器
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

def preprocess_and_split_with_overlap(text, max_length=256, stride=128):
    """
    使用滑動窗口切割長文本，保留一定的上下文重疊。
    
    Args:
        text (str): 待切割的文本。
        max_length (int): 每個段落的最大 token 長度。
        stride (int): 滑動窗口的步長，重疊部分長度 = max_length - stride。
        
    Returns:
        list: 切割後的文本段列表。
    """
    tokens = tokenizer.encode(text, truncation=False)  # 不截斷，獲取所有 tokens
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]  # 當前窗口的 tokens
        chunks.append(chunk)
        if end == len(tokens):  # 最後一段退出
            break
        start += stride  # 滑動窗口開始位置
    # 將 tokens 解碼回文本
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# 初始化列表存放處理後的數據
processed_data = []

# 讀取 CSV 文件
df = pd.read_csv(csv_file, encoding="ISO-8859-1")

# 遍歷 DataFrame 中的每行並顯示進度條
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {csv_file}"):
    pdf_name = row['PDF']        # PDF 文件名稱
    page_number = row['Page ']   # 注意列名中的空格（根據CSV內容調整）
    core_terms = row['Core Terms']  # 需要處理的文本內容
    
    # 對內容進行處理和滑動窗口切割（修正變量名）
    chunks = preprocess_and_split_with_overlap(core_terms)
    
    # 保存分塊後的數據（保存chunk而非原始內容）
    for chunk in chunks:
        processed_data.append({
            'PDF': pdf_name,
            'Page': page_number,
            'Core Terms Chunk': chunk  # 存儲分塊後的內容
        })

# 將處理後的數據轉為 DataFrame 並保存
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv('Processed_CoreTerms_Data.csv', index=False)

print("處理完成，已保存為 'Processed_CoreTerms_Data.csv'")

Processing /Users/tangjiahong/Dropbox/textmining1/Data_Clean/CSV/merge_CSV/merge_CoreTerms.csv: 100%|██████████| 9259/9259 [00:10<00:00, 907.38it/s]

處理完成，已保存為 'Processed_CoreTerms_Data.csv'


In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 加載微調後的模型
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_model")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_model")

# 加載知識圖譜
with open(r"C:\Users\User\Dropbox\textmining1\KG\definition_moreinfo.json", "r", encoding="utf-8-sig") as f:
    knowledge_graph = json.load(f)

# 建立名稱到定義的映射
name_to_definition = {item["Name"]: item["Definition"] for item in knowledge_graph}

print(f"Loaded knowledge graph with {len(knowledge_graph)} entries.")

# 定義檢索知識圖譜的函數
def retrieve_from_knowledge_graph(answer, knowledge_graph):
    """
    根據模型的答案在知識圖譜中進行檢索。
    """
    related_definitions = [entry for entry in knowledge_graph if entry["Name"].lower() in answer.lower()]
    return related_definitions if related_definitions else [{"Name": "No Match", "Definition": "No related definition found."}]

# 定義 RAG 測試管線
def rag_test_pipeline(row, model, tokenizer, knowledge_graph):
    """
    單條測試管線，適配切割後的資料。
    """
    pdf_file = row['PDF']  # PDF 文件名稱
    page_number = row['Page']  # 頁碼
    label = row['Label']  # 標籤
    paragraph = row['Text']
    
    # 定義測試問題
    question = f"What is the judge's opinion regarding entry barriers in case from {pdf_file}, page {page_number}?"
    
    # 預測答案
    inputs = tokenizer.encode_plus(question, paragraph, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"].tolist()[0]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    
    # 檢索知識圖譜
    related_definitions = retrieve_from_knowledge_graph(answer, knowledge_graph)
    
    return {
        "PDF": pdf_file,
        "Page": page_number,
        "Label": label,
        "Question": question,
        "Predicted Answer": answer,
        "Knowledge Graph Result": related_definitions
    }

# 批量處理的預測函數
def batch_predict(model, tokenizer, questions, contexts, batch_size=16):
    """
    批量處理問題和上下文，進行預測
    """
    predictions = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for i in tqdm(range(0, len(questions), batch_size), desc="Predicting"):
        batch_questions = questions[i:i + batch_size]
        batch_contexts = contexts[i:i + batch_size]
        
        inputs = tokenizer(batch_questions, batch_contexts, return_tensors="pt", 
                           padding=True, truncation=True, max_length=512).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        for j in range(len(batch_questions)):
            start = torch.argmax(outputs.start_logits[j]).item()
            end = torch.argmax(outputs.end_logits[j]).item() + 1
            input_ids = inputs["input_ids"][j].tolist()
            prediction = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(input_ids[start:end])
            )
            predictions.append(prediction)
    
    return predictions

# 加載處理後的數據
valid_data = pd.read_csv("Processed_CoreTerms_Data.csv")

# 提取問題和上下文
questions = [f"What is the judge's opinion regarding entry barriers in case from {pdf}, page {page}?"
             for pdf, page in zip(valid_data["PDF"], valid_data["Page"])]
contexts = valid_data["Core Terms Chunk"].tolist()

print(f"Loaded {len(questions)} samples for validation.")

# 執行批量預測
predicted_answers = batch_predict(model, tokenizer, questions, contexts)



Loaded knowledge graph with 13 entries.
Loaded 9259 samples for validation.


Predicting: 100%|██████████| 579/579 [00:39<00:00, 14.57it/s]


KeyError: 'Label'

In [17]:
# 生成結果
def generate_results(valid_data, questions, contexts, predicted_answers, knowledge_graph):
    """
    將驗證數據和模型預測結果封裝成包含 PDF 和頁碼的結果列表。
    """
    results = []
    for pdf, page, question, context, prediction in zip(
        valid_data["PDF"], valid_data["Page"], questions, contexts, predicted_answers):
        
        related_definitions = retrieve_from_knowledge_graph(prediction, knowledge_graph)
        results.append({
            "PDF": pdf,
            "Page": page,
            "Question": question,
            "Context": context,
            "Predicted Answer": prediction,
            "Knowledge Graph Result": related_definitions
        })
    return results

# 調用函數生成 results
results = generate_results(valid_data, questions, contexts, predicted_answers, knowledge_graph)


In [19]:
# 將結果保存為 JSON 文件
import json
# 將結果保存為 JSON 文件
with open("RAG_Test_Results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Results saved to 'RAG_Test_Results.json'")

# 查看部分結果
for result in results[:5]:
    print("PDF:", result["PDF"])
    print("Page:", result["Page"])
    print("Question:", result["Question"])
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)


Results saved to 'RAG_Test_Results.json'
PDF: a1  
Page: 21
Question: What is the judge's opinion regarding entry barriers in case from a1  , page 21?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1  
Page: 65
Question: What is the judge's opinion regarding entry barriers in case from a1  , page 65?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1  
Page: 73
Question: What is the judge's opinion regarding entry barriers in case from a1  , page 73?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1  
Page: 81
Question: What is the judge's opinion regarding entry barriers in case from a1  , page 81?
Predicted Answer: [SEP]
Knowledge Graph Result: [{'Name': 'No Match', 'Definition': 'No related definition found.'}]
PDF: a1  
Page: 85
Question: What is the judge's op

In [21]:
# 載入保存的結果
import json

# 加載結果
with open("RAG_Test_Results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# 過濾有匹配的結果
matched_results = [
    result for result in results if result["Knowledge Graph Result"][0]["Name"] != "No Match"
]

# 列出有匹配的結果
print(f"Found {len(matched_results)} matched results.")
for result in matched_results:
    print("PDF:", result["PDF"])
    print("Page:", result["Page"])
    print("Question:", result["Question"])
    print("Context:", result["Context"][:500], "...")
    print("Predicted Answer:", result["Predicted Answer"])
    print("Knowledge Graph Result:", result["Knowledge Graph Result"])
    print("=" * 50)

# 列出匹配的結果數量
print(f"Found {len(matched_results)} matched results.")

# 保存匹配結果到新文件
output_file = "Matched_CoreTerms_DATA.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(matched_results, f, ensure_ascii=False, indent=4)


Found 1 matched results.
PDF: c5  
Page: 308
Question: What is the judge's opinion regarding entry barriers in case from c5  , page 308?
Context: newspaper, prices, independent dealer, monopoly power, plaintiffs ', home delivery, monopolize, vertical integration, circulation, coercion, monopoly, conspiracy, antitrust, dealers, sales, sherman act, customers, delivery, retail, defendants ', wholesalers, adherence, attempt to monopolize, relevant market, specific intent, probability, summary judgment, publisher ...
Predicted Answer: [SEP] newspaper , prices , independent dealer , monopoly power , plaintiffs ' , home delivery , monopolize , vertical integration , circulation , coercion , monopoly , conspiracy , antitrust , dealers , sales , sherman act , customers , delivery , retail , defendants ' , wholesalers , adherence , attempt to monopolize , relevant market , specific intent , probability , summary judgment , publisher [SEP]
Knowledge Graph Result: [{'Name': 'Vertical Integration',